In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('../../')

In [6]:
import json
import os
from glob import glob
import numpy as np
import pandas as pd

from utils.parse_util import HTMLFileReader, LinkedInProfileParser
from utils.parse_pipeline import parse_profile
import webbrowser

In [7]:
LOG_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/parse_log/First_1000_1'
PARSED_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/parsed_profiles/First_1000_0'

### Log

In [51]:
def read_log_file():

    log_fpaths = glob(os.path.join(LOG_DIR, '*.json'))
    for fpath in log_fpaths:
        with open(fpath, 'r') as f:
            content = json.load(f)
        yield fpath, content

In [52]:
log_file_contents = read_log_file()

In [53]:
log_items = []

for fpath, log_content in log_file_contents:
    log_item = log_content.copy()
    log_item['empty_html_list'] = len(log_item['empty_html_list'])
    log_item['parse_error'] = len(log_item['parse_error'])
    log_items.append(log_item)

In [54]:
log_items = pd.DataFrame(log_items)

In [55]:
log_items.shape

(324, 4)

In [56]:
temp = log_items.sum()

In [57]:
(temp / temp['num_files']) * 100

empty_html_list      0.000260
parse_error          0.018916
total_time           0.037398
num_files          100.000000
dtype: float64

### Sample Log

In [137]:
log_file = '122_log.json'
log_path = os.path.join(LOG_DIR, log_file)
log_path

'/home/sjb/Projects/Research/LinkedIn_OB/data/parse_log/First_1000_0/122_log.json'

In [138]:
with open(log_path, 'r') as f:
    log_content = json.load(f)

In [139]:
log_content.keys()

dict_keys(['empty_html_list', 'parse_error', 'total_time', 'num_files'])

In [140]:
log_content['parse_error']

{'/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6050100.html': 'Expected child node to be not None',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6056720.html': 'Expected child node to be not None',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6056732.html': 'Expected child node to be not None',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6062517.html': 'Expected child node to be not None',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6092758.html': 'Expected child node to be not None'}

In [141]:
errored_files = list(log_content['parse_error'].keys())
errored_files

['/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6050100.html',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6056720.html',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6056732.html',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6062517.html',
 '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/122/6092758.html']

#### Error file check

In [142]:
webbrowser.open('file://' + errored_files[4])

True

In [143]:
html_reader = HTMLFileReader(errored_files[0])
html_reader.load()
clean_html = html_reader.get_clean_html_text()

In [144]:
li_profile = LinkedInProfileParser(clean_html)
li_profile.parse()

ValueError: Expected child node to be not None

In [ ]:
li

### Parsed Dirs

In [181]:
subdirs = os.listdir(PARSED_DIR)

In [182]:
len(subdirs)

500

### Parsed Profile

In [191]:
parsed_file_paths = glob(os.path.join(PARSED_DIR, '*/*.json'))

In [192]:
len(parsed_file_paths)

2644957

In [148]:
sample_parsed_fp = np.random.choice(parsed_file_paths)

In [149]:
with open(sample_parsed_fp, 'r') as f:
    sample_parse_content = json.load(f)

In [150]:
sample_parse_content.keys()

dict_keys(['header', 'overview', 'summary', 'specialty', 'experience', 'education'])

In [151]:
sample_parse_content['summary']

'Writer, editor, and web consultant with extensive training and experience in both the language arts and computer science.'